In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import logging
import pickle
from joblib import Parallel, delayed
import gc
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [2]:
base_path = './data'
feature_path = './feature'

In [3]:
# 话题向量
# topic_table = pd.read_csv(f'{base_path}/topic_vectors_64d.txt', sep='\t', header=None)
# topic_table.columns = ['topic', 'vec']

# def str2vec(s):
#     tmp = s.split(' ')
#     res = []
#     for num in tmp:
#         res.append(float(num))
#     return res

# topic_table['vec'] = topic_table['vec'].apply(str2vec)

# import pickle
# pickle.dump(topic_table, open(f'{base_path}/topic_vec.pkl', 'wb'))

In [4]:
topic_table = pickle.load(open(f'{base_path}/topic_vec.pkl', 'rb'))

In [5]:
# 问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t', usecols=[0, 6])
ques.columns = ['qid', 'topic']

logging.info("ques %s", ques.shape)

[2019-12-06 16:18:35,790] INFO in <ipython-input-5-26315363bfb1>: ques (1829900, 2)


In [6]:
# 回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t', usecols=[0, 1, 2, 3])
ans.columns = ['aid', 'qid', 'uid', 'ans_dt']
logging.info("ans %s", ans.shape)

[2019-12-06 16:19:37,128] INFO in <ipython-input-6-48fd985f6174>: ans (4513735, 4)


In [7]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))

ans['day'] = extract_day(ans['ans_dt'])
del ans['ans_dt']
logging.info("ans %s", ans.shape)

[2019-12-06 16:19:52,898] INFO in <ipython-input-7-2a4889d719c3>: ans (4513735, 4)


In [8]:
# ans 对应的问题的话题
ans = pd.merge(ans, ques, on='qid', how='left')

In [9]:
# 4折统计
def fold_fn(x):
    if 3838<=x<=3846:
        return 0
    if 3847<=x<=3853:
        return 1
    if 3854<=x<=3860:
        return 2
    if 3861<=x<=3867:
        return 3
    else:
        return -1     # 更前的一个月

In [10]:
# train
ans['fold'] = ans['day'].apply(fold_fn)

In [11]:
ans_with_topic = ans[ans['topic']!='-1']

In [12]:
user_topic_0 = ans_with_topic[ans_with_topic['fold']!=0].groupby('uid')['topic'].agg(','.join)
logging.info('fold 0, user num: %s', len(user_topic_0))

user_topic_1 = ans_with_topic[ans_with_topic['fold']!=1].groupby('uid')['topic'].agg(','.join)
logging.info('fold 1, user num: %s', len(user_topic_1))

user_topic_2 = ans_with_topic[ans_with_topic['fold']!=2].groupby('uid')['topic'].agg(','.join)
logging.info('fold 2, user num: %s', len(user_topic_2))

user_topic_3 = ans_with_topic[ans_with_topic['fold']!=3].groupby('uid')['topic'].agg(','.join)
logging.info('fold 3, user num: %s', len(user_topic_3))

[2019-12-06 16:21:09,293] INFO in <ipython-input-12-bfcd1724525a>: fold 0, user num: 701146
[2019-12-06 16:22:03,747] INFO in <ipython-input-12-bfcd1724525a>: fold 1, user num: 693487
[2019-12-06 16:22:58,837] INFO in <ipython-input-12-bfcd1724525a>: fold 2, user num: 693658
[2019-12-06 16:23:51,056] INFO in <ipython-input-12-bfcd1724525a>: fold 3, user num: 666274


In [42]:
# test
# user_topic_all = ans.groupby('uid')['topic'].agg(','.join)  # 给 test 用
# pickle.dump(user_topic_all, open(f'{base_path}/user_topic.pkl', 'wb'))

In [51]:
# 压缩数据
def compress_data(df):
    t = df.dtypes
    for x in t[t == 'int64'].index:
        df[x] = df[x].astype('int32')

    for x in t[t == 'float64'].index:
        df[x] = df[x].astype('float32')
    return df

In [52]:
def cos_sim(vec1, vec2):
    return np.dot(vec1, vec2)/((np.linalg.norm(vec1)*(np.linalg.norm(vec2))))

def eucl_sim(vec1, vec2):
    return np.linalg.norm(np.array(vec1)-np.array(vec2))

In [53]:
# 用户在其他时期回答的话题统计特征
def user_topic_stat(u_topic):
    n_most_common = 20     # 截取回答次数前面的话题
        
    if u_topic == '-1':
        return ([0] + [np.nan]*15)
    
    u_topic_2 = u_topic.split(',')
    counter = Counter(u_topic_2)          
    most_common_topic = counter.most_common(n_most_common)
    
    u_topic_most_com = []     # 前几个最常回答的话题
    count_list = []           # 这几个常回答的话题对应的计数
    for ut_c in most_common_topic: 
        u_topic_most_com.append(ut_c[0])
        count_list.append(ut_c[1])
    count_norm_list = np.array(list(count_list)) / np.sum(list(count_list))   # 这几个常回答话题对应的计数比例
    
    """
    用户话题的统计
    """
    # count_list 的统计信息
    count_u_topic = len(count_list)  
    mean_u_topic = np.mean(count_list)
    std_u_topic = np.std(count_list)  
    sum_u_topic = np.sum(count_list)
    max_u_topic = np.max(count_list)
    min_u_topic = np.min(count_list)
    # res 1
    res = [count_u_topic, mean_u_topic, std_u_topic, sum_u_topic, max_u_topic, min_u_topic]
    
    if len(most_common_topic)>1:   # 2 以上，计算两两相似度
        sim_list = []
        sim_list_eucl = []
        for i in range(len(most_common_topic)):
            for j in range(i+1, len(most_common_topic)):
                ut1_index = int(u_topic_most_com[i][1:]) - 1
                ut2_index = int(u_topic_most_com[j][1:]) - 1
                ut1 = topic_table.iloc[ut1_index]['vec']
                ut2 = topic_table.iloc[ut2_index]['vec']

                sim_list.append(cos_sim(ut1, ut2))
                sim_list_eucl.append(eucl_sim(ut1, ut2))
        # 余弦相似度
        min_uu_sim = np.min(sim_list)
        max_uu_sim = np.max(sim_list)
        sum_uu_sim = np.sum(sim_list)
        mean_uu_sim = np.mean(sim_list)
        std_uu_sim = np.std(sim_list)
        # 欧氏距离
        min_uu_sim_eucl = np.min(sim_list_eucl)
        max_uu_sim_eucl = np.max(sim_list_eucl)
        sum_uu_sim_eucl = np.sum(sim_list_eucl)
        mean_uu_sim_eucl = np.mean(sim_list_eucl)
        std_uu_sim_eucl = np.std(sim_list_eucl)
        # res 2
        res += [min_uu_sim, max_uu_sim, sum_uu_sim, mean_uu_sim, std_uu_sim]
        res += [min_uu_sim_eucl, max_uu_sim_eucl, sum_uu_sim_eucl, mean_uu_sim_eucl, std_uu_sim_eucl]
    else:       # 用户回答个数为 1
        # res 2
        res += [np.nan] * 10
    
    return res

In [63]:
import traceback

processes = 36   # 设置进程数

def target_fn(series, path, num):
    try:
        logging.info('start to extract feature.')
        df_from_series = series.reset_index()
        t1 = df_from_series.apply(lambda x: user_topic_stat(x['topic']),axis=1, result_type='expand')
        col_name = ['count_u_topic', 'mean_u_topic', 'std_u_topic', 'sum_u_topic', 'max_u_topic', 'min_u_topic', 
                    'min_uu_sim', 'max_uu_sim', 'sum_uu_sim', 'mean_uu_sim', 'std_uu_sim',
                    'min_uu_sim_eucl', 'max_uu_sim_eucl', 'sum_uu_sim_eucl', 'mean_uu_sim_eucl', 'std_uu_sim_eucl']
        t1.columns = col_name
        res = pd.concat([df_from_series, t1], axis=1)
        logging.info('extracting finish.')
        
        # 压缩数据
        res = compress_data(res)
        res.to_csv(f'{path}/user_topic_feature_{num}.txt', index=False, sep='\t')
        logging.info('file %s saving finish.', num)
        del res
        gc.collect()
    except:
        print(traceback.print_exc())
        
def multi_proc_1(series, frag_path):
    import multiprocessing
    pool = multiprocessing.Pool(processes=processes)
    len_data = len(series)
    len_batch = len_data // processes
    for i in range(processes):
        start = i * len_batch
        end = (i+1) * len_batch
        if i == (processes-1):
            end = len_data
        tmp = series[start:end]
        pool.apply_async(target_fn, (tmp, frag_path, i))
        
    pool.close()
    pool.join()

In [64]:
multi_proc_1(user_topic_0, './temp_user_topic_stat/fold0')
multi_proc_1(user_topic_1, './temp_user_topic_stat/fold1')
multi_proc_1(user_topic_2, './temp_user_topic_stat/fold2')
multi_proc_1(user_topic_3, './temp_user_topic_stat/fold3')

[2019-12-07 13:37:02,613] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:02,728] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:03,130] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:03,328] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:03,517] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:03,709] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:03,964] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:04,124] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:04,242] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:04,328] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 13:37:04,416] INFO in <ipython-input-63-835ceec1a953>: sta

[2019-12-07 14:04:58,221] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:04:58,903] INFO in <ipython-input-63-835ceec1a953>: file 12 saving finish.
[2019-12-07 14:05:14,546] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:05:15,244] INFO in <ipython-input-63-835ceec1a953>: file 33 saving finish.
[2019-12-07 14:05:17,209] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:05:17,850] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:05:17,925] INFO in <ipython-input-63-835ceec1a953>: file 7 saving finish.
[2019-12-07 14:05:18,549] INFO in <ipython-input-63-835ceec1a953>: file 8 saving finish.
[2019-12-07 14:05:52,995] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:05:53,682] INFO in <ipython-input-63-835ceec1a953>: file 6 saving finish.
[2019-12-07 14:06:22,348] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:06:22,853] INFO in <ip

[2019-12-07 14:33:19,399] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:20,072] INFO in <ipython-input-63-835ceec1a953>: file 10 saving finish.
[2019-12-07 14:33:27,175] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:27,867] INFO in <ipython-input-63-835ceec1a953>: file 34 saving finish.
[2019-12-07 14:33:38,905] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:39,664] INFO in <ipython-input-63-835ceec1a953>: file 30 saving finish.
[2019-12-07 14:33:43,678] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:44,364] INFO in <ipython-input-63-835ceec1a953>: file 14 saving finish.
[2019-12-07 14:33:46,048] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:47,438] INFO in <ipython-input-63-835ceec1a953>: file 12 saving finish.
[2019-12-07 14:33:52,178] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:33:52,856] INFO in 

[2019-12-07 14:59:10,661] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:59:11,404] INFO in <ipython-input-63-835ceec1a953>: file 14 saving finish.
[2019-12-07 14:59:57,236] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 14:59:57,946] INFO in <ipython-input-63-835ceec1a953>: file 6 saving finish.
[2019-12-07 14:59:59,605] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:00:00,338] INFO in <ipython-input-63-835ceec1a953>: file 5 saving finish.
[2019-12-07 15:00:16,727] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:00:17,446] INFO in <ipython-input-63-835ceec1a953>: file 7 saving finish.
[2019-12-07 15:02:37,444] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:02:38,135] INFO in <ipython-input-63-835ceec1a953>: file 31 saving finish.
[2019-12-07 15:03:21,100] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:03:21,865] INFO in <ip

[2019-12-07 15:27:49,863] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:27:50,665] INFO in <ipython-input-63-835ceec1a953>: file 17 saving finish.
[2019-12-07 15:28:20,088] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:28:20,849] INFO in <ipython-input-63-835ceec1a953>: file 16 saving finish.
[2019-12-07 15:28:21,684] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:28:22,364] INFO in <ipython-input-63-835ceec1a953>: file 26 saving finish.
[2019-12-07 15:28:29,745] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:28:30,516] INFO in <ipython-input-63-835ceec1a953>: file 12 saving finish.
[2019-12-07 15:28:48,773] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:28:49,441] INFO in <ipython-input-63-835ceec1a953>: file 10 saving finish.
[2019-12-07 15:29:04,282] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 15:29:04,964] INFO in 

In [65]:
# 合数据
def concat_data(original_path, dest_path):
    topic_feat = None
    for i in range(processes):
        d = pd.read_csv(f'{original_path}/user_topic_feature_{i}.txt', sep='\t')
        if topic_feat is None:
            topic_feat = d
        else:
            topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
            
    logging.info('topic feature, shape: %s, start saving', topic_feat.shape)
    
    topic_feat = compress_data(topic_feat)
    topic_feat.to_csv(f'{dest_path}/topic_feature.txt', index=False, sep='\t')
    logging.info('saving done. From %s to %s', original_path, dest_path)

In [66]:
concat_data('./temp_user_topic_stat/fold0', f'./temp_user_topic_stat/dest0')
concat_data('./temp_user_topic_stat/fold1', f'./temp_user_topic_stat/dest1')
concat_data('./temp_user_topic_stat/fold2', f'./temp_user_topic_stat/dest2')
concat_data('./temp_user_topic_stat/fold3', f'./temp_user_topic_stat/dest3')

[2019-12-07 15:36:43,216] INFO in <ipython-input-65-9b27b2ae3db8>: topic feature, shape: (701146, 18), start saving
[2019-12-07 15:37:31,016] INFO in <ipython-input-65-9b27b2ae3db8>: saving done. From ./temp_user_topic_stat/fold0 to ./temp_user_topic_stat/dest0
[2019-12-07 15:37:38,882] INFO in <ipython-input-65-9b27b2ae3db8>: topic feature, shape: (693487, 18), start saving
[2019-12-07 15:38:26,013] INFO in <ipython-input-65-9b27b2ae3db8>: saving done. From ./temp_user_topic_stat/fold1 to ./temp_user_topic_stat/dest1
[2019-12-07 15:38:33,786] INFO in <ipython-input-65-9b27b2ae3db8>: topic feature, shape: (693658, 18), start saving
[2019-12-07 15:39:07,312] INFO in <ipython-input-65-9b27b2ae3db8>: saving done. From ./temp_user_topic_stat/fold2 to ./temp_user_topic_stat/dest2
[2019-12-07 15:39:10,516] INFO in <ipython-input-65-9b27b2ae3db8>: topic feature, shape: (666274, 18), start saving
[2019-12-07 15:39:27,050] INFO in <ipython-input-65-9b27b2ae3db8>: saving done. From ./temp_user_t

In [76]:
pickle.dump(user_topic_0, open(f'{base_path}/user_topic_fold_0.pkl', 'wb'))
pickle.dump(user_topic_1, open(f'{base_path}/user_topic_fold_1.pkl', 'wb'))
pickle.dump(user_topic_2, open(f'{base_path}/user_topic_fold_2.pkl', 'wb'))
pickle.dump(user_topic_3, open(f'{base_path}/user_topic_fold_3.pkl', 'wb'))

In [13]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
train['day'] = extract_day(train['dt'])
del train['dt']
logging.info("train %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
del test['dt']
logging.info("test %s", test.shape)

[2019-12-06 16:24:46,468] INFO in <ipython-input-13-229aa93b02cb>: train (9489162, 4)
[2019-12-06 16:24:49,139] INFO in <ipython-input-13-229aa93b02cb>: test (1141683, 2)


In [14]:
train = pd.merge(train, ques, on='qid', how='left')
test = pd.merge(test, ques, on='qid', how='left')

In [15]:
# train
train['fold'] = train['day'].apply(fold_fn)

train['user_topic_kfold'] = -10000

for i in range(4):
    train.loc[train['fold']==i, 'user_topic_kfold'] = train[train['fold']==i]['uid'].map(ut_list[i]).fillna('-1')

assert len(train[train['user_topic_kfold']==-10000])==0

In [17]:
# test
user_topic = pickle.load(open(f'{base_path}/user_topic.pkl', 'rb'))       # 没有打折
test['user_topic_kfold'] = test['uid'].map(user_topic).fillna('-1')    # 实际并不是 kfold

In [67]:
user_topic

uid
M1000000382                                             T13,T567
M1000000983    T6192,T1667,T2255,T1103,T70,T100,T20647,T1878,...
M1000008978                                T1891,T56,T5,T39,T131
M1000020034                                                 T597
M1000022555                             T1440,T608,T103,T13,T103
                                     ...                        
M999984680     T26,T277,T76,T245,T542,T2274,T9526,T51857,T200...
M999988985     T3259,T11393,T2231,T3928,T2229,T1814,T1257,T72...
M999995457     T2,T488,T11088,T6248,T355,T7752,T674,T2,T22,T3...
M999998695                      T810,T5568,T5568,T810,T5568,T810
M999998888                       T35,T296,T4979,T3,T162,T67,T296
Name: topic, Length: 767269, dtype: object

In [68]:
multi_proc_1(user_topic, './temp_user_topic_stat/all')
concat_data('./temp_user_topic_stat/all', f'./temp_user_topic_stat/all_dest')

[2019-12-07 15:39:32,226] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,275] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,318] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,364] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,404] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,444] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,488] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,529] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,571] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,613] INFO in <ipython-input-63-835ceec1a953>: start to extract feature.
[2019-12-07 15:39:32,649] INFO in <ipython-input-63-835ceec1a953>: sta

[2019-12-07 16:10:45,051] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:10:45,795] INFO in <ipython-input-63-835ceec1a953>: file 5 saving finish.
[2019-12-07 16:10:46,114] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:10:46,882] INFO in <ipython-input-63-835ceec1a953>: file 4 saving finish.
[2019-12-07 16:11:07,647] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:11:08,400] INFO in <ipython-input-63-835ceec1a953>: file 1 saving finish.
[2019-12-07 16:11:09,666] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:11:10,413] INFO in <ipython-input-63-835ceec1a953>: file 7 saving finish.
[2019-12-07 16:12:09,061] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:12:09,805] INFO in <ipython-input-63-835ceec1a953>: file 2 saving finish.
[2019-12-07 16:12:13,103] INFO in <ipython-input-63-835ceec1a953>: extracting finish.
[2019-12-07 16:12:13,855] INFO in <ipyt

In [70]:
# 问题话题与用户在其他时期回答的话题交叉特征
def qu_topic_sim(q_topic, u_topic):
    n_most_common = 20     # 截取回答次数前面的话题
        
    if (q_topic == '-1' or u_topic == '-1'):
        return ([0]*4 + [np.nan]*20)
    
    q_topic_2 = q_topic.split(',')
    u_topic_2 = u_topic.split(',')
    counter = Counter(u_topic_2)          
    most_common_topic = counter.most_common(n_most_common)
    
    u_topic_most_com = []     # 前几个最常回答的话题
    count_list = []           # 这几个常回答的话题对应的计数
    for ut_c in most_common_topic: 
        u_topic_most_com.append(ut_c[0])
        count_list.append(ut_c[1])
    count_norm_list = np.array(list(count_list)) / np.sum(list(count_list))   # 这几个常回答话题对应的计数比例
    
    """
    问题话题与用户话题交互统计
    """
    
    count_ut_in_qt_weighted = 0     # 问题话题在过往回答过的话题里的出现次数
    count_ut_in_qt = 0             # 问题话题与过往回答过的话题的交集个数
    count_norm_ut_in_qt = 0
    # 余弦相似度
    sim_list = []                   # 两两交叉计算相似度
    sim_norm_list = []
    # 欧氏距离
    sim_list_eucl = []                   # 两两交叉计算相似度
    sim_norm_list_eucl = []

    for qt in q_topic_2:
        q_topic_index = int(qt[1:]) - 1   # q_topic 索引
        q_topic_vec = topic_table.iloc[q_topic_index]['vec']
        for ut_c, count, count_norm in zip(most_common_topic, count_list, count_norm_list):  
            u_topic_index = int(ut_c[0][1:]) - 1   # u_topic 索引
            u_topic_vec = topic_table.iloc[u_topic_index]['vec']
            if q_topic_index == u_topic_index:
                count_ut_in_qt_weighted += ut_c[1]
                count_ut_in_qt += 1
                count_norm_ut_in_qt += count_norm
            
            sim = cos_sim(q_topic_vec, u_topic_vec)
            sim_norm = sim*count_norm
            sim_eucl = eucl_sim(q_topic_vec, u_topic_vec)
            sim_norm_eucl = sim_eucl*count_norm
            
            sim_list.append(sim)
            sim_norm_list.append(sim_norm)
            sim_list_eucl.append(sim_eucl)
            sim_norm_list_eucl.append(sim_norm_eucl)
            
    # 出现在问题和回答过的问题的话题，在问题话题中所占比例
    rate_ut_in_qt = count_ut_in_qt / len(q_topic_2)
    
    # 余弦相似度
    min_qu_sim = np.min(sim_list)
    max_qu_sim = np.max(sim_list)
    sum_qu_sim = np.sum(sim_list)
    mean_qu_sim = np.mean(sim_list)
    std_qu_sim = np.std(sim_list)
    min_qu_sim_norm = np.min(sim_norm_list)
    max_qu_sim_norm = np.max(sim_norm_list)
    sum_qu_sim_norm = np.sum(sim_norm_list)
    mean_qu_sim_norm = np.mean(sim_norm_list)
    std_qu_sim_norm = np.std(sim_norm_list)
    # 欧氏距离
    min_qu_sim_eucl = np.min(sim_list_eucl)
    max_qu_sim_eucl = np.max(sim_list_eucl)
    sum_qu_sim_eucl = np.sum(sim_list_eucl)
    mean_qu_sim_eucl = np.mean(sim_list_eucl)
    std_qu_sim_eucl = np.std(sim_list_eucl)
    min_qu_sim_norm_eucl = np.min(sim_norm_list_eucl)
    max_qu_sim_norm_eucl = np.max(sim_norm_list_eucl)
    sum_qu_sim_norm_eucl = np.sum(sim_norm_list_eucl)
    mean_qu_sim_norm_eucl = np.mean(sim_norm_list_eucl)
    std_qu_sim_norm_eucl = np.std(sim_norm_list_eucl)
    
    # 问题话题与用户回答话题的统计
    res = [count_ut_in_qt_weighted, count_ut_in_qt, rate_ut_in_qt, count_norm_ut_in_qt]
    res += [min_qu_sim, max_qu_sim, sum_qu_sim, mean_qu_sim, std_qu_sim]
    res += [min_qu_sim_norm, max_qu_sim_norm, sum_qu_sim_norm, mean_qu_sim_norm, std_qu_sim_norm]
    res += [min_qu_sim_eucl, max_qu_sim_eucl, sum_qu_sim_eucl, mean_qu_sim_eucl, std_qu_sim_eucl]
    res += [min_qu_sim_norm_eucl, max_qu_sim_norm_eucl, sum_qu_sim_norm_eucl, mean_qu_sim_norm_eucl, std_qu_sim_norm_eucl]
    return res

In [71]:
import traceback

processes = 36   # 设置进程数

def cross_qu_topic_feat(df, num, mode):
    try:
        logging.info('start to extract feature.')
        t1 = df.apply(lambda x: qu_topic_sim(x['topic'], x['user_topic_kfold']), axis=1, result_type='expand')
        col_name = ['qu_topic_count_weight', 'qu_topic_count', 'qu_topic_rate', 'qu_topic_count_norm',
                  'min_sim', 'max_sim', 'sum_sim', 'mean_sim', 'std_sim', 'min_sim_norm', 
                  'max_sim_norm', 'sum_sim_norm', 'mean_sim_norm', 'std_sim_norm', 
                  'min_sim_eucl', 'max_sim_eucl', 'sum_sim_eucl', 'mean_sim_eucl', 'std_sim_eucl',
                  'min_sim_norm_eucl', 'max_sim_norm_eucl', 'sum_sim_norm_eucl', 'mean_sim_norm_eucl', 'std_sim_norm_eucl']
        t1.columns = col_name
        logging.info('extracting finish.')
        
        # 压缩数据
        t1 = compress_data(t1)
        t1.to_csv(f'./temp/{mode}_kfold_topic_feature_{num}.txt', index=False, sep='\t')
        logging.info('file %s saving finish.', num)
        del t1
        gc.collect()
    except:
        print(traceback.print_exc())
        
def multi_proc(df, mode):
    import multiprocessing
    pool = multiprocessing.Pool(processes=processes)
    len_data = len(df)
    len_batch = len_data // processes
    for i in range(processes):
        start = i * len_batch
        end = (i+1) * len_batch
        if i == (processes-1):
            end = len_data
        tmp = df[start:end]
        pool.apply_async(cross_qu_topic_feat, (tmp, i, mode))
        
    pool.close()
    pool.join()

In [ ]:
multi_proc(test, 'test')

[2019-12-07 16:25:37,458] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:37,633] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:37,752] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:37,687] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:37,891] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:37,973] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:38,063] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:38,185] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:38,268] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:38,363] INFO in <ipython-input-71-00f3ed632dd9>: start to extract feature.
[2019-12-07 16:25:38,440] INFO in <ipython-input-71-00f3ed632dd9>: sta

In [ ]:
multi_proc(train, 'train')

In [28]:
# 合数据

# test
topic_feat = None
original_path = './temp'
mode = 'test'
for i in range(processes):
    d = pd.read_csv(f'{original_path}/{mode}_kfold_topic_feature_{i}.txt', sep='\t')
    if topic_feat is None:
        topic_feat = d
    else:
        topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
        
logging.info('%s topic feature, shape: %s', mode, topic_feat.shape)

topic_feat = compress_data(topic_feat)
topic_feat.to_csv(f'{feature_path}/{mode}_kfold_topic_feature.txt', index=False, sep='\t')
logging.info('%s topic feature saved.', mode)

[2019-12-06 21:27:22,347] INFO in <ipython-input-28-ec6b0f27b465>: test topic feature, shape: (1141683, 24)
[2019-12-06 21:28:43,531] INFO in <ipython-input-28-ec6b0f27b465>: test topic feature saved.


In [29]:
# train
topic_feat = None
original_path = './temp'
mode = 'train'
for i in range(processes):
    d = pd.read_csv(f'{original_path}/{mode}_kfold_topic_feature_{i}.txt', sep='\t')
    if topic_feat is None:
        topic_feat = d
    else:
        topic_feat = pd.concat([topic_feat, d], axis=0, ignore_index=True)
logging.info('%s topic feature, shape: %s', mode, topic_feat.shape)

topic_feat = compress_data(topic_feat)
topic_feat.to_csv(f'{feature_path}/{mode}_kfold_topic_feature.txt', index=False, sep='\t')
logging.info('%s topic feature saved.', mode)

[2019-12-06 21:29:58,898] INFO in <ipython-input-29-1fe54e2cf7eb>: train topic feature, shape: (9489162, 24)
[2019-12-06 21:40:33,107] INFO in <ipython-input-29-1fe54e2cf7eb>: train topic feature saved.
